In [ ]:
import os
import random
import warnings

# Elaborazione Dati e Calcolo Scientifico
import numpy as np
import pandas as pd
import cv2

# Visualizzazione
import seaborn as sns
import matplotlib.pyplot as plt

# Deep Learning (TensorFlow/Keras)
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from classification_models.tfkeras import Classifiers

# Machine Learning Utility (Scikit-learn)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

# Configurazione ambiente
warnings.filterwarnings('ignore')

In [ ]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

In [ ]:
!pip install image-classifiers

In [ ]:
# Updated to match your directory listing
ROOT_DIR = "/content/GroceryStoreDataset/dataset/"
classes_df = pd.read_csv(os.path.join(ROOT_DIR, "classes.csv"))
print(f"Shape: {classes_df.shape}")
print(f"Unique Classes: {classes_df['Class ID (int)'].unique()}")
classes_df.head()

def process_dataframe_fixed(df, root_dir):
    df['path'] = df['path'].apply(lambda x: os.path.join(root_dir, x))
    # Convertiamo ogni singola cella in stringa
    df['fine_label'] = df['fine_label'].astype(str)
    return df
    
# Ricarica i dati per pulire errori precedenti
train_df = pd.read_csv(os.path.join(ROOT_DIR, "train.txt"), header=None, sep=",", names=['path', 'fine_label', 'coarse_label'])
val_df   = pd.read_csv(os.path.join(ROOT_DIR, "val.txt"), header=None, sep=",", names=['path', 'fine_label', 'coarse_label'])
test_df  = pd.read_csv(os.path.join(ROOT_DIR, "test.txt"), header=None, sep=",", names=['path', 'fine_label', 'coarse_label'])
train_df = process_dataframe_fixed(train_df, ROOT_DIR)
val_df = process_dataframe_fixed(val_df, ROOT_DIR)
test_df = process_dataframe_fixed(test_df, ROOT_DIR)
print(f"Classi uniche in Train: {train_df['fine_label'].nunique()}")
print(f"Classi uniche in Val:   {val_df['fine_label'].nunique()}")
print(f"Classi uniche in Test:  {test_df['fine_label'].nunique()}")

In [ ]:
df_combined = pd.concat([train_df, val_df], ignore_index=True)
lista_classi = sorted(df_combined['fine_label'].unique().tolist())
NUM_CLASSES = len(lista_classi)

train_df_new, val_df_new = train_test_split(
    df_combined,
    test_size=0.20,
    stratify=df_combined['fine_label'],
    random_state=42
)

In [ ]:
IMG_RAW_H, IMG_RAW_W = 256, 256
IMG_CROP_H, IMG_CROP_W = 224, 224
CHANNELS = 3
BATCH_SIZE = 64
EPOCHS = 60

# ImageDataGenerator per il caricamento (solo rescaling)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df_new,
    x_col='path',
    y_col='fine_label',
    target_size=(IMG_RAW_H, IMG_RAW_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=lista_classi,
    shuffle=True
)
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df_new,
    x_col='path',
    y_col='fine_label',
    target_size=(IMG_RAW_H, IMG_RAW_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=lista_classi,
    shuffle=False
)
# Calcolo pesi delle classi
train_labels = train_generator.classes
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
dict_weights = dict(enumerate(weights))

In [ ]:
def build_final_model(input_shape_raw, crop_height, crop_width, num_classes):
    # Definizione esplicita dell'input
    inputs = layers.Input(shape=input_shape_raw)

    # DATA AUGMENTATION (Attiva solo in fase di training)
    x = layers.RandomCrop(crop_height, crop_width)(inputs)
    #x = layers.RandomFlip("horizontal")(x)
    #x = layers.RandomRotation(0.2)(x)
    #x = layers.RandomContrast(0.1)(x)
    x = layers.RandomFlip("horizontal")(x)
    x = layers.RandomRotation(0.2)(x)
    x = layers.RandomContrast(0.1)(x)
    x = layers.RandomZoom(0.1)(x)
    x = layers.RandomTranslation(0.1,0.1)(x)
    
    # Blocchi Convoluzionali (Simil-VGG ma con BatchNormalization e Dropout)
    # Blocco 1
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Blocco 2
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Blocco 3
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Blocco 4
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Classifier
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.02))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

In [ ]:
model = build_final_model(
    input_shape_raw=(IMG_RAW_H, IMG_RAW_W, CHANNELS),
    crop_height=IMG_CROP_H,
    crop_width=IMG_CROP_W,
    num_classes=NUM_CLASSES
)
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0004),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)
# Callback (EarlyStopping e Learning Rate Decay)
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks_list,
    class_weight=dict_weights
)

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 5))

    # Grafico Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Grafico Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()
    
def plot_final_evaluation(model, generator, title="Validation Set"):
    print(f"\n--- Valutazione su: {title} ---")

    # 1. Reset del generatore (fondamentale per mantenere l'ordine)
    generator.reset()

    # 2. Ottenere le predizioni
    y_pred_probs = model.predict(generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = generator.classes

    # 3. Calcolo e visualizzazione Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 12))
    sns.heatmap(cm, annot=False, cmap='Blues')
    plt.title(f'Confusion Matrix ({title})')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    # 4. Classification Report
    class_labels = list(generator.class_indices.keys())
    print(f"\n--- Classification Report ({title}) ---")
    print(classification_report(y_true, y_pred, target_names=class_labels))


plot_history(history)

plot_final_evaluation(model, val_generator, title="Validation Set")

plot_final_evaluation(model, test_generator, title="Test Set")

REST-NET tuning

In [ ]:
ResNet18, preprocess_input = Classifiers.get('resnet18')

# =========================
# 2. GENERATORI DATI
# =========================
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df_new,
    x_col='path',
    y_col='fine_label',
    target_size=(IMG_RAW_H, IMG_RAW_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=lista_classi,
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df_new,
    x_col='path',
    y_col='fine_label',
    target_size=(IMG_RAW_H, IMG_RAW_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=lista_classi,
    shuffle=False
)

# =========================
# 3. CLASS WEIGHTS
# =========================
train_labels = train_generator.classes
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
dict_weights = dict(enumerate(weights))


In [ ]:

def build_resnet18_advanced(num_classes, learning_rate, fine_tune=False):

    inputs = layers.Input(shape=(IMG_RAW_H, IMG_RAW_W, 3), name="input_image")

    # ---- Augmentation interna ----
    x = layers.RandomCrop(IMG_CROP_H, IMG_CROP_W)(inputs)
    x = layers.RandomFlip("horizontal")(x)
    x = layers.RandomRotation(0.2)(x)
    x = layers.RandomContrast(0.1)(x)
    x = layers.RandomZoom(0.1)(x)
    x = layers.RandomTranslation(0.1,0.1)(x)

    # ---- Base Model ----
    base_model = ResNet18(
        weights='imagenet',
        include_top=False,
        input_tensor=x
    )
    base_model.trainable = fine_tune

    if fine_tune:
        for layer in base_model.layers:
            if isinstance(layer, layers.BatchNormalization):
                layer.trainable = False

    x = base_model.output

    # ---- Classification Head ----
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.02))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    return model

In [ ]:

# =========================
# 4. COSTRUZIONE MODELLO
# =========================
# =========================
# 5. CALLBACKS
# =========================
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

# =========================
# 6. TRAINING WARM-UP
# =========================
tf.keras.backend.clear_session()
print("\n>>> FASE 1: Warm-up (Base Congelata)")
model = build_resnet18_advanced(NUM_CLASSES, learning_rate=1e-3, fine_tune=False)

history_warmup = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    class_weight=dict_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            restore_best_weights=True,
            verbose=1
        )
    ]
)

# =========================
# 7. FINE-TUNING
# =========================
print("\n>>> FASE 2: Fine-tuning (Base Sbloccata)")
weights_warmup = model.get_weights()
model = build_resnet18_advanced(NUM_CLASSES, learning_rate=1e-5, fine_tune=True)
model.set_weights(weights_warmup)

history_finetune = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    class_weight=dict_weights,
    callbacks=callbacks_list
)

In [ ]:

# =========================
# 8. VALIDAZIONE COMPLETA
# =========================
def plot_history(history, title="Training History"):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(title + " - Accuracy")
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(title + " - Loss")
    plt.legend()
    plt.show()

plot_history(history_warmup, title="Warm-up")
plot_history(history_finetune, title="Fine-tuning")

# Classification report + confusion matrix
val_generator.reset()
y_pred = model.predict(val_generator, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred_classes, target_names=lista_classi))

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, fmt='d', cmap='Blues')
plt.title("Confusion Matrix - Validation Set")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# =========================
# 9. TEST GENERATOR & EVALUATION
# =========================
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='path',
    y_col='fine_label',
    target_size=(IMG_RAW_H, IMG_RAW_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=lista_classi,
    shuffle=False
)

test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")